![alt text](Kung_Men.jpg)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
# base_path = "/kaggle/input/playground-series-s4e10"
df_train = pd.read_csv(os.path.join('train.csv'))
df_test = pd.read_csv(os.path.join('test.csv'))
df_sample = pd.read_csv(os.path.join('sample_submission.csv'))
TARGET = 'loan_status'



Preprocessing

In [2]:
for column in df_train.columns:
    nan_count = df_train[column].isna().sum()
    print(f"Column '{column}' has {nan_count} NaN values.")


Column 'id' has 0 NaN values.
Column 'person_age' has 0 NaN values.
Column 'person_income' has 0 NaN values.
Column 'person_home_ownership' has 0 NaN values.
Column 'person_emp_length' has 0 NaN values.
Column 'loan_intent' has 0 NaN values.
Column 'loan_grade' has 0 NaN values.
Column 'loan_amnt' has 0 NaN values.
Column 'loan_int_rate' has 0 NaN values.
Column 'loan_percent_income' has 0 NaN values.
Column 'cb_person_default_on_file' has 0 NaN values.
Column 'cb_person_cred_hist_length' has 0 NaN values.
Column 'loan_status' has 0 NaN values.


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in df_train.select_dtypes(include=['object']).columns:
    df_train[col] = label_encoder.fit_transform(df_train[col])

for col in df_test.select_dtypes(include=['object']).columns:
    df_test[col] = label_encoder.fit_transform(df_test[col])

print(df_train.head())


   id  person_age  person_income  person_home_ownership  person_emp_length  \
0   0          37          35000                      3                0.0   
1   1          22          56000                      2                6.0   
2   2          29          28800                      2                8.0   
3   3          30          70000                      3               14.0   
4   4          22          60000                      3                2.0   

   loan_intent  loan_grade  loan_amnt  loan_int_rate  loan_percent_income  \
0            1           1       6000          11.49                 0.17   
1            3           2       4000          13.35                 0.07   
2            4           0       6000           8.90                 0.21   
3            5           1      12000          11.11                 0.17   
4            3           0       6000           6.92                 0.10   

   cb_person_default_on_file  cb_person_cred_hist_length  loan_statu

In [4]:
print(df_train.describe())

                 id    person_age  person_income  person_home_ownership  \
count  58645.000000  58645.000000   5.864500e+04           58645.000000   
mean   29322.000000     27.550857   6.404617e+04               1.673578   
std    16929.497605      6.033216   3.793111e+04               1.452534   
min        0.000000     20.000000   4.200000e+03               0.000000   
25%    14661.000000     23.000000   4.200000e+04               0.000000   
50%    29322.000000     26.000000   5.800000e+04               3.000000   
75%    43983.000000     30.000000   7.560000e+04               3.000000   
max    58644.000000    123.000000   1.900000e+06               3.000000   

       person_emp_length   loan_intent    loan_grade     loan_amnt  \
count       58645.000000  58645.000000  58645.000000  58645.000000   
mean            4.701015      2.519430      1.066638   9217.556518   
std             3.959784      1.722896      1.046181   5563.807384   
min             0.000000      0.000000      

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = df_train.drop(['id','loan_status'], axis=1).values
# X_scaled = scaler.fit_transform(df_train.drop(['id','loan_status'], axis=1))

y_train = df_train['loan_status'].values

# X_test = df_test.values

X_test_ids = df_test['id'] 
X_test_scaled = scaler.transform(df_test.drop('id', axis=1))  # Scale the test set features



![alt text](DeepNN.jpg)

In [10]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

clf1 = LogisticRegression()
clf2 = SVC(probability=True)
# clf3 = DecisionTreeClassifier()
clf4 = GradientBoostingClassifier(max_depth = 3, n_estimators=100)
clf5 = AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)

# Create Voting Classifier
# voting_clf = VotingClassifier(estimators=[
#     ('lr', clf1), ('svc', clf2), ('dt', clf3), ('gbt', clf4)], voting='soft')
voting_clf = VotingClassifier(estimators=[
    ('lr', clf1), ('svc', clf2), ('AdaBoost', clf5), ('gbt', clf4)], voting='soft')
# Train ensemble
voting_clf.fit(X_train, y_train)

# Make predictions
y_pred = voting_clf.predict(X_valid)
print("Accuracy:", roc_auc_score(y_valid, y_pred))



# Create the model
# model = Sequential()
# model.add(Dense(4, input_dim=X_scaled.shape[1], activation='relu'))  # First hidden layer
# model.add(Dense(4, activation='relu'))  # Second hidden layer
# model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
# model.compile(loss='binary_crossentropy', optimizer='sgd')

# Fit the model
# model.fit(X_scaled, y_train, epochs=100, batch_size=10, verbose=1)


c:\Users\Big_Guppy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.8331550298219552


In [42]:

# Make predictions
y_test_pred_probs = voting_clf.predict(X_test_scaled).flatten()

# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'id': X_test_ids,
    'loan_status': y_test_pred_probs
})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)



1222/1222 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
